In [3]:
import keras
from keras import models,layers

In [4]:
from keras import backend
from keras.datasets import mnist

In [6]:
# 데이터 획득
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [8]:
# 이미지 크기
img_rows, img_cols = x_train.shape[1:]
img_rows, img_cols

(28, 28)

In [9]:
# 채널의 위치 조정
backend.image_data_format()

'channels_last'

In [10]:
if backend.image_data_format() == 'channel_first':
    x_train = x_train.reshape( x_train.shape[0], 1, img_rows, img_cols )
    x_test  = x_test.reshape(  x_test.shape[0],  1, img_rows, img_cols  )
    input_shape = ( 1, img_rows, img_cols )
else:
    # 흑백 이미지의 채널 정보를 처리하기 위해서는 추가 차원을 넣어야 한다
    # 컬러 이미지는 RGB색상을 다루는 채널 정보가 이미지 데이터에 포함되어 있다
    # 흑백은 채널 정보가 업서엇, 입력 데이터 차원을 하나 더 추가해야 하다
    # 채널의 위치를 확인해서 앞쪽이면 앞으로, 뒤쪽이면 뒤로 배치
    x_train = x_train.reshape( x_train.shape[0], img_rows, img_cols, 1  )
    x_test  = x_test.reshape(  x_test.shape[0],  img_rows, img_cols, 1  )
    input_shape = ( img_rows, img_cols, 1 )

In [12]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

In [13]:
x_train /= 255
x_test  /= 255

In [14]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [20]:
a = set(y_train)
label_nums = len(a)

In [21]:
# 레이블의 분류형 변수로 특성 이진화 처리
y_train = keras.utils.to_categorical( y_train, label_nums )
y_test  = keras.utils.to_categorical( y_test, label_nums )

In [24]:
y_train[:2]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [23]:
x_train.shape, y_train.shape

((60000, 28, 28, 1), (60000, 10))

In [25]:
# 모델 생성
model = models.Sequential()

In [26]:
# 합성곱층 생성
# 32: 출력 채널
# 커널, 필터, 가중치 => 3x3, 활성화함수 지정, 편향은 생략
model.add( layers.Conv2D( 32, kernel_size=(3,3), 
                          activation='relu', input_shape=input_shape ) )

Instructions for updating:
Colocations handled automatically by placer.


In [27]:
# 풀링층
model.add( layers.MaxPool2D( pool_size=(2,2) ) )

In [28]:
# 과적합 방지
# 0.25 : 설정값
model.add( layers.Dropout( 0.25 ) )

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
# 합성곱 2
model.add( layers.Conv2D( 64, (3,3), activation='relu' ) )

In [30]:
# 풀링층 2
model.add( layers.MaxPool2D( pool_size=(2,2) ) )

In [31]:
# 과적합 방지
# 0.25 : 설정값
model.add( layers.Dropout( 0.25 ) )

In [32]:
# 데이터를 편다 
model.add( layers.Flatten() )

In [33]:
# 전결합층
# 128: 출력 채널 설정
model.add( layers.Dense( 128, activation='relu' ) )

In [34]:
# 과적합 방지
# 0.5 : 설정값
model.add( layers.Dropout( 0.5 ) )

In [35]:
# 출력층
model.add( layers.Dense( label_nums, activation='softmax' ) )

In [36]:
# 손실함수:크로스엔트로피 
# 최적화:평균제곱근오차 : rmsprop
# 정확도
model.compile( loss=keras.losses.categorical_crossentropy,
               optimizer='rmsprop', metrics=['accuracy'])

In [37]:
# 학습 사이즈 
batch_size = 128
# 수행회수
epochs     = 10

In [38]:
# 훈련데이터 중에 검증용 20% 사용
his = model.fit( x_train, y_train, batch_size=batch_size, epochs=epochs, 
                 validation_split=0.2 )

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 110s 2ms/step - loss: 0.3447 - acc: 0.8921 - val_loss: 0.0873 - val_acc: 0.9752
Epoch 2/10
48000/48000 [==============================] - 67s 1ms/step - loss: 0.1151 - acc: 0.9657 - val_loss: 0.0639 - val_acc: 0.9813
Epoch 3/10
48000/48000 [==============================] - 66s 1ms/step - loss: 0.0879 - acc: 0.9745 - val_loss: 0.0471 - val_acc: 0.9865
Epoch 4/10
48000/48000 [==============================] - 68s 1ms/step - loss: 0.0725 - acc: 0.9775 - val_loss: 0.0407 - val_acc: 0.9887
Epoch 5/10
48000/48000 [==============================] - 79s 2ms/step - loss: 0.0630 - acc: 0.9815 - val_loss: 0.0371 - val_acc: 0.9897
Epoch 6/10
48000/48000 [==============================] - 76s 2ms/step - loss: 0.0584 - acc: 0.9822 - val_loss: 0.0354 - val_acc: 0.9899
Epoch 7/10
48000/48000 [==============================] - 73s 2ms/step - loss: 

In [40]:
score = model.evaluate( x_test, y_test )
score

10000/10000 [==============================] - 4s 435us/step


[0.025114613091268983, 0.9914]